In [1]:
import re
import shutil
import pandas as pd
from pandas import DataFrame, Series
import github3
from github3 import login
from git import Repo, GitCommandError
import yaml
from subprocess import Popen, PIPE, STDOUT, SubprocessError, check_output
from configparser import ConfigParser
from github3.repos.branch import Branch
from github3.null import NullObject
from schematics.models import Model, BaseType
from schematics.types import StringType, IntType, BooleanType
from schematics.types.compound import ListType, DictType
from schematics.exceptions import ValidationError
%cd ~/Documents/projects/stitch/python/
from stitch.core.stitch_frame import StitchFrame
%cd ~/Documents/projects/nerve/python/
from nerve.core.model import Nerve
from nerve.core.client import Client
from nerve.core.utils import *
from nerve.core.git import Git
from nerve.core.git_lfs import GitLFS
from nerve.core.metadata import Metadata
from nerve.core.utils import execute_subprocess, get_status
from nerve.core.api import *
from nerve.spec.specifications import *
from nerve.spec.validators import *
from nerve.spec.traits import *
from nerve.spec import traits
from nerve.test.test_suite import *
%cd /tmp/ntr002

/Users/alex/Documents/projects/stitch/python
/Users/alex/Documents/projects/nerve/python
/private/tmp/ntr002


In [3]:
admin = NerveAdmin()
info = admin._nerve._get_info('ntr002')
info.project

{'deliverables': ['vol001', 'geo001'],
 'gitignore': ['__pycache__/',
  '*.py[cod]',
  '*.so',
  '.Python',
  'env/',
  'build/',
  'develop-eggs/',
  'dist/',
  'downloads/',
  'eggs/',
  '.eggs/',
  'lib/',
  'lib64/',
  'parts/',
  'sdist/',
  'var/',
  '*.egg-info/',
  '.installed.cfg',
  '*.egg',
  '*.manifest',
  '*.spec',
  'pip-log.txt',
  'htmlcov/',
  '.tox/',
  '.coverage',
  '.coverage.*',
  '.cache',
  'nosetests.xml',
  'coverage.xml',
  '*,cover',
  '*.mo',
  '*.pot',
  '*.log',
  'docs/_build/',
  'target/',
  '.ipynb_checkpoints',
  '.DS_Store',
  'node_modules/'],
 'lfs-extensions': ['obj', 'png', 'mb'],
 'nondeliverables': ['maya001'],
 'notes': '',
 'private': True,
 'project-name': 'ntr002',
 'specification': 'proj001',
 'teams': {'biz': 'read', 'dev': 'write'},
 'version': None}

In [4]:
admin.delete('ntr002', True, True)

True

In [5]:
admin.create('ntr002', 'some note from the admin about the project')
admin.clone('ntr002')

True

In [6]:
%%bash
rm -rf /tmp/ntr002-new
cp -R /tmp/ntr002/ /tmp/ntr002-old
cd /tmp/ntr002/
mkdir vol001/ntr002_vol001_desc_v001_-_-_-
echo v001 sdjfkh 0000 > vol001/ntr002_vol001_desc_v001_-_-_-/ntr002_vol001_desc_v001_-_-_0000.png
echo v001 jyelkh 0001 > vol001/ntr002_vol001_desc_v001_-_-_-/ntr002_vol001_desc_v001_-_-_0001.png
echo v001 lwekfj 0002 > vol001/ntr002_vol001_desc_v001_-_-_-/ntr002_vol001_desc_v001_-_-_0002.png
echo vasc v001 > maya001/ntr002_maya001_vasc_v001.mb
echo kidney v001 > maya001/ntr002_maya001_kdny_v001.mb
tree

.
├── geo001
├── maya001
│   ├── ntr002_maya001_kdny_v001.mb
│   └── ntr002_maya001_vasc_v001.mb
├── ntr002_proj001_meta.yml
└── vol001
    └── ntr002_vol001_desc_v001_-_-_-
        ├── ntr002_vol001_desc_v001_-_-_0000.png
        ├── ntr002_vol001_desc_v001_-_-_0001.png
        └── ntr002_vol001_desc_v001_-_-_0002.png

4 directories, 6 files


In [4]:
%%bash
cd /tmp/ntr002/
mkdir vol001/ntr002_vol001_desc_v002_-_-_-
echo v002 sdjfkh 0000 > vol001/ntr002_vol001_desc_v002_-_-_-/ntr002_vol001_desc_v002_-_-_0000.png
echo v002 jyelkh 0001 > vol001/ntr002_vol001_desc_v002_-_-_-/ntr002_vol001_desc_v002_-_-_0001.png
echo v002 lwekfj 0002 > vol001/ntr002_vol001_desc_v002_-_-_-/ntr002_vol001_desc_v002_-_-_0002.png
echo vasc v002 > maya001/ntr002_maya001_vasc_v002.mb
echo kidney v002 > maya001/ntr002_maya001_kdny_v002.mb
tree

In [7]:
os.chdir('/tmp/ntr002')
stat = list(admin.status())
data = None
if len(stat) > 0:
    data = DataFrame(list(stat)).applymap(lambda x: x.data)
    data = StitchFrame(data).flatten(prefix=False).to_dataframe().T
data

,0,1,2
asset-id,9b1838a8-1f4e-47aa-9584-51...,b15b4690-cb1f-4c23-aa48-55...,03c11c2e-fad2-419c-b5e3-f1...
asset-name,ntr002_maya001_kdny_v001,ntr002_maya001_vasc_v001,ntr002_vol001_desc_v001_-_-_-
asset-type,nondeliverable,nondeliverable,deliverable
data,None,None,None
dependencies,NaN,NaN,[]
descriptor,NaN,NaN,desc
notes,,,
project-id,73967966,73967966,73967966
project-name,ntr002,ntr002,ntr002
project-url,git@github.com:nerve-test-...,git@github.com:nerve-test-...,git@github.com:nerve-test-...


In [8]:
admin.publish('some note from the alex', verbosity=2)

True

In [9]:
%%bash
mv /tmp/ntr002 /tmp/ntr002-new
mv /tmp/ntr002-old/ /tmp/ntr002

In [10]:
admin = NerveAdmin()
admin.request()

True